In [1]:
import pandas as pd
import re
from datetime import datetime




In [ ]:

file_path = "C:\\Users\\Aditya Tripathi\\Finothon\\Cleaned_SMS_Data.csv"


sms_data = pd.read_csv(file_path)


sms_data['text'] = sms_data['text'].fillna('').astype(str)


In [ ]:

def analyze_transaction(text):

    spam_keywords = ['bonus', 'cashback', 'offer', 'app', 'win', 'prize', 'pool', 'click here', 'contest','claim']
    
    if any(keyword in text.lower() for keyword in spam_keywords):
        return None, None, None  # Skip this message
    

    transaction_type = None
    platform = None
    payment_method = None
    

    debit_keywords = r'\b(paid|charged|debited|withdrawn|deducted|spent|transferred|EMI|settled|fee|disbursed|purchase)\b'
    credit_keywords = r'\b(credited|received|refunded|reversed|deposited|added|reimbursed|awarded|bonus|loan approved|cashback|interest earned|payment received|gift|processed)\b'
    

    if re.search(debit_keywords, text, re.IGNORECASE):
        transaction_type = "Paid/Debited"
    elif re.search(credit_keywords, text, re.IGNORECASE):
        transaction_type = "Credited"


    platform_match = re.search(r'on\s([\w\s]+?)\s(charged|paid|via)', text, re.IGNORECASE)
    if platform_match:
        platform = platform_match.group(1).strip()


    payment_method_match = re.search(r'via\s([\w\s]+)', text, re.IGNORECASE)
    if payment_method_match:
        payment_method = payment_method_match.group(1).strip()

    return transaction_type, platform, payment_method


sms_data[['transaction_type', 'platform', 'payment_method']] = sms_data['text'].apply(
    lambda x: pd.Series(analyze_transaction(x))
)

In [ ]:

def extract_amount(text)
    
    amounts = re.findall(r'(₹|Rs\.?)\s?(\d+[.,]?\d*)', text)
    if amounts:
       
        amount = float(amounts[0][1].replace(',', ''))
       
        if amount > 10:  
            return amount
    return None

def classify_loan(text):
    
    loan_keywords = r'\b(loan|loan amount|loan approved|personal loan|instant approval)\b'
    
    return bool(re.search(loan_keywords, text, re.IGNORECASE))




sms_data['amount'] = sms_data['text'].apply(extract_amount)


sms_data = sms_data[sms_data['transaction_type'].notnull() & sms_data['amount'].notnull()]


sms_data['debited_amount'] = sms_data['amount'].where(sms_data['transaction_type'] == 'Paid/Debited', 0)
sms_data['credited_amount'] = sms_data['amount'].where(sms_data['transaction_type'] == 'Credited', 0)


sms_data['total_amount'] = sms_data['debited_amount'] + sms_data['credited_amount']


sms_data['is_loan_related'] = sms_data['text'].apply(classify_loan)


sms_data['day'] = pd.to_datetime(sms_data['updateAt'], errors='coerce').dt.day
sms_data['month'] = pd.to_datetime(sms_data['updateAt'], errors='coerce').dt.month
sms_data['year'] = pd.to_datetime(sms_data['updateAt'], errors='coerce').dt.year
sms_data['time'] = pd.to_datetime(sms_data['updateAt'], errors='coerce').dt.time


sms_data.drop(columns=['updateAt'], inplace=True)


output_path = 'Report 3.csv'
sms_data.to_csv(output_path, index=False)

print(f"Updated transaction reports saved to {output_path}")


In [ ]:
import pandas as pd

file_path = "C:\\Users\\Aditya Tripathi\\Finothon\\Report 1.csv
sms_data = pd.read_csv(file_path)

sms_data['senderAddress'] = sms_data['senderAddress'].fillna('').astype(str)

def extract_payment_method(sender):
  
    if "-" in sender:
        return sender.split("-")[1].lower()  # Convert to lowercase for uniformity
    return sender.lower()
sms_data['payment_method'] = sms_data['senderAddress'].apply(extract_payment_method)

output_path = "C:\\Users\\Aditya Tripathi\\Finothon\\Updated_SMS_Data.csv"
sms_data.to_csv(output_path, index=False)

print(f"Updated SMS data with 'payment_method' column saved to {output_path}")